# Create timetable calendar from Excel spreadsheet

This code takes a named cell range in an Excel spreadsheet and produces a ics calendar file.

In [11]:
import numpy as np
import pandas as pd
import openpyxl
import icalendar
from icalendar import Calendar, Event, vText, vCalAddress

Documentation for iCalendar library can be found at: http://icalendar.readthedocs.io/en/latest/

In [12]:
import phjReadData

import importlib
importlib.reload(phjReadData)

<module 'phjReadData' from '/Users/philjones/Dropbox/phjPythonModules/PHJ-GitHub/phj_python_modules/phjReadData.py'>

## Enter paths and filenames

In [13]:
phjPath = '/Users/philjones/Documents/Teaching/Liverpool/New curriculum/Academic year 2016-17/Timetables'
phjExcelFilename = 'EPHW draft timetable 2016-17 as at 2016-09-01.xlsx'
phjExcelRangeName = 'vsci100_ephw_sem1'

# The following will be used to label calendar events
phjCourseHeader = 'VSCI100 EPHW 2016-17'

# The following file will be created
phjICSFilename = phjCourseHeader + ' calendar.ics'

## Read data from Excel spreadsheet

In [14]:
phjTimetableDF = phjReadData.phjReadDataFromExcelNamedCellRange(phjExcelFileName = '/'.join([phjPath,phjExcelFilename]),
                                                                phjExcelRangeName = phjExcelRangeName,
                                                                phjDatetimeFormat = "%Y-%m-%d",
                                                                phjMissingValue = np.nan,
                                                                phjHeaderRow = True,
                                                                phjPrintResults = True)


List of iterable properties:
['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', 'destinations', 'localSheetId', 'name', 'repr_format', 'scope', 'str_format', 'value']

List of tuples of named ranges and cell ranges
[(<ReadOnlyWorksheet "Sheet1">, '$A$5:$L$109')]

First tuple of named ranges and cell range
(<ReadOnlyWorksheet "Sheet1">, '$A$5:$L$109')

Cell range
$A$5:$L$109

Local Sheet ID
<ReadOnlyWorksheet "Sheet1">

First row (containing variable names) has been removed from the data.
var 1 :  date
var 2 :  day_number
var 3 :  week_number
var 4 :  time_string
var 5 :  start_time
var 6 :  end_time
var 7 :  number_hours
var 8 :  room
var 9 :  type
var 10 :  title
var 11 :  staff
var 12 :  comments

Imported data
-----

In [15]:
phjTimetableDF['date'] = pd.to_datetime(phjTimetableDF['date'])

Remove rows where no time_string information is set (i.e. no lecture takeing place)

In [16]:
phjTimetableDF = phjTimetableDF[phjTimetableDF['time_string'].notnull()].reset_index(drop = True)
print(phjTimetableDF)

         date  day_number  week_number            time_string start_time  \
0  2016-10-11           2            3     4:00 pm to 5:00 pm    4:00 pm   
1  2016-10-12           3            3   11:00 am to 12:00 pm   11:00 am   
2  2016-10-14           5            3   10:00 am to 11:00 am   10:00 am   
3  2016-10-18           2            4     3:00 pm to 4:00 pm    3:00 pm   
4  2016-10-19           3            4  11:00 am to 12:00  pm   11:00 am   
5  2016-10-20           4            4    9:00 am to 10:00 am    9:00 am   
6  2016-10-20           4            4   10:00 am to 11:00 am   10:00 am   
7  2016-10-20           4            4  11:00 am to 12:00  pm   11:00 am   
8  2016-10-21           5            4    9:00 am to 10:00 am    9:00 am   
9  2016-10-24           1            5  10:00 am to 12:00  pm   10:00 am   
10 2016-10-24           1            5     2:00 pm to 4:00 pm    2:00 pm   
11 2016-10-24           1            5     3:00 pm to 5:00 pm    3:00 pm   
12 2016-10-2

In [17]:
cal = Calendar()

In [18]:
cal.add('prodid', '-//Python iCalendar timetable//')
cal.add('version', '2.0')
organizer = vCalAddress('MAILTO:p.h.jones@liverpool.ac.uk')
organizer.params['cn'] = vText('Phil Jones')
organizer.params['role'] = vText('Lecturer')

# UID will be (arbitrarily) constucted using datetime of start event, title
# of event and the label given below to provide human-readable code.
# Actually, by creating a UID that will be duplicated if the script is re-run,
# the old items in the calendar will be replaced if the .ics file is added.
# Therefore, didn't create a UID using datetime of 'now'.
# Equally, it is important that different event that overlap don't have the
# same UiD, hence include the title of the event to ensure UIDs are different.
phjEventUIDLabel = 'vsci100_2016-17_ephw_sem1@liv.ac.uk'

for index, row in phjTimetableDF.iterrows():
    
    # Create event in calendar object
    event = Event()
    
    # Add properties to event
    event.add('organizer', organizer)
    
    # See above note about creating UIDs for events
    event.add('uid', '/'.join([pd.to_datetime(row['date'].strftime('%Y-%m-%d') + ' ' + row['start_time']).strftime('%Y%m%d%H%M%S'),
                               row['title'],
                               phjEventUIDLabel]))
    
    if pd.isnull(row['staff']):
        event.add('summary', phjCourseHeader + ' ' + row['type'] + ' - ' + row['title'])
    else:
        event.add('summary', phjCourseHeader + ' ' + row['type'] + ' - ' + row['title'] + ' (' + row['staff'] + ')')
        
    event.add('dtstart', pd.Timestamp(row['date'].strftime('%Y-%m-%d') + ' ' + row['start_time'], tz='Europe/London'))
    event.add('dtend',   pd.Timestamp(row['date'].strftime('%Y-%m-%d') + ' ' + row['end_time'], tz='Europe/London'))
    event.add('location', row['room'])
    
    # Add event to calendar
    cal.add_component(event)

In [19]:
print(cal.to_ical())

b'BEGIN:VCALENDAR\r\nVERSION:2.0\r\nPRODID:-//Python iCalendar timetable//\r\nBEGIN:VEVENT\r\nSUMMARY:VSCI100 EPHW 2016-17 LECTURE - Introduction to VPH (Eleni Michalop\r\n oulou)\r\nDTSTART;TZID=Europe/London;VALUE=DATE-TIME:20161011T160000\r\nDTEND;TZID=Europe/London;VALUE=DATE-TIME:20161011T170000\r\nUID:20161011160000/Introduction to VPH/vsci100_2016-17_ephw_sem1@liv.ac.uk\r\nLOCATION:TBA\r\nORGANIZER;CN="Phil Jones";ROLE=Lecturer:MAILTO:p.h.jones@liverpool.ac.uk\r\nEND:VEVENT\r\nBEGIN:VEVENT\r\nSUMMARY:VSCI100 EPHW 2016-17 LECTURE - Introduction to state veterinary me\r\n dicine (Jim Scudamore)\r\nDTSTART;TZID=Europe/London;VALUE=DATE-TIME:20161012T110000\r\nDTEND;TZID=Europe/London;VALUE=DATE-TIME:20161012T120000\r\nUID:20161012110000/Introduction to state veterinary medicine/vsci100_2016-\r\n 17_ephw_sem1@liv.ac.uk\r\nLOCATION:TBA\r\nORGANIZER;CN="Phil Jones";ROLE=Lecturer:MAILTO:p.h.jones@liverpool.ac.uk\r\nEND:VEVENT\r\nBEGIN:VEVENT\r\nSUMMARY:VSCI100 EPHW 2016-17 LECTURE - In

Write ical of all events in timetable to file in same location as original Excel file

In [20]:
f = open('/'.join([phjPath,phjICSFilename]),'wb')
f.write(cal.to_ical())
f.close()